# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [416]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [417]:
# File to Load (Remember to Change These)
city_data_load = "WeatherPy/WeatherPy.csv"
#student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
city_df = pd.read_csv(city_data_load)
#student_data = pd.read_csv(student_data_to_load)
city_df.dropna(inplace = True) 
city_df

,City,Country,Max Temp,Date,Cloudiness,Lat,Lng,Wind Speed,Humidity
0,George Town,MY,75.20,1619481639,40,5.4112,100.3354,1.14,94
1,Vila Velha,BR,78.80,1619481639,40,-20.3297,-40.2925,2.30,89
2,Dosso,NE,89.49,1619481363,81,13.0490,3.1937,4.32,27
3,Rikitea,PF,78.06,1619481640,72,-23.1203,-134.9692,7.99,72
4,Itarana,BR,71.74,1619481640,58,-19.8739,-40.8753,3.49,88
...,...,...,...,...,...,...,...,...,...
533,Zeya,RU,49.35,1619481795,98,53.7500,127.2667,13.09,58
534,Bālurghāt,IN,79.63,1619481795,87,25.2167,88.7667,4.03,14
535,Jackson,US,60.01,1619481502,90,42.4165,-122.8345,9.22,41
536,Byron Bay,AU,68.00,1619481658,40,-28.6500,153.6167,9.22,77


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [418]:
# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = city_df["Humidity"].astype(float)

In [419]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [420]:
city1 = city_df.loc[(city_df["Max Temp"] < 80) & (city_df["Max Temp"] > 70)]
city2 = city1.loc[(city1["Wind Speed"] < 10)]
city3 = city2.loc[city2["Cloudiness"] == 0]
city3

,City,Country,Max Temp,Date,Cloudiness,Lat,Lng,Wind Speed,Humidity
110,Bignona,SN,75.20,1619481667,0,12.8103,-16.2264,4.61,73
458,Conde,BR,77.00,1619481771,0,-7.2597,-34.9075,5.75,94
496,Inhambane,MZ,71.53,1619481783,0,-23.8650,35.3833,5.79,88
505,Akdepe,TM,71.60,1619481786,0,42.0551,59.3788,6.91,41


In [421]:
#city4 = city3.loc[(city1["Wind Speed"] > 5)]
#city4

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [422]:
hotel_df = pd.DataFrame({})
hotel_df


hotel_df["Hotel Name"]= ""
hotel_df["City"] = city3["City"]
hotel_df["Country"]=city3["Country"]
hotel_df["Lat"] = ""
hotel_df["Lng"]=""


hotel_df

,Hotel Name,City,Country,Lat,Lng
110,NaN,Bignona,SN,,
458,NaN,Conde,BR,,
496,NaN,Inhambane,MZ,,
505,NaN,Akdepe,TM,,


In [423]:
for index, row in city3.iterrows():
   

    lat = city3.loc[index,"Lat"]
    lng = city3.loc[index,"Lng"]
    target_radius = 5000
    target_type = "hotel"
    
    
    params = {
    "location": f"{lat},{lng}",
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
   
# base url
   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
# run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
 
    geo = results[0]["geometry"]
    loc = geo["location"]
    lt = loc["lat"]
    ln = loc["lng"]
# Print the json (pretty printed)
    #print(json.dumps(results, indent=4, sort_keys=False))
    
    print(f"Retrieving Results for Index {index}")
    print(results[0]["name"])
    hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    hotel_df.loc[index, 'Lat'] = loc["lat"]
    hotel_df.loc[index, 'Lng'] = loc["lng"]
    
    print("======================")
    
        
        

Retrieving Results for Index 110
Bignona
Retrieving Results for Index 458
João Pessoa
Retrieving Results for Index 496
Maxixe
Retrieving Results for Index 505
Akdepe


In [424]:
hotel_df

,Hotel Name,City,Country,Lat,Lng
110,Bignona,Bignona,SN,12.8051,-16.2346
458,João Pessoa,Conde,BR,-7.1195,-34.845
496,Maxixe,Inhambane,MZ,-23.8606,35.3468
505,Akdepe,Akdepe,TM,42.0532,59.3846


In [425]:
# geocoordinates
#target_coordinates = "-9.6658,-35.7353"
print(target_coordinates)
#target_coordinates = city3["Lat"],city3["Lng"]
target_radius = 3000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
#response = requests.get(base_url, params=params).json()



-9.6658,-35.7353


In [426]:
# convert response to json
#places_data = response.json()
#results = response['results']
# Print the json (pretty printed)
#print(json.dumps(results, indent=4, sort_keys=False))

In [427]:
#print(results[0]["name"])
#print(results[0]["geometry"])
#g = results[0]["geometry"]
#a = g["location"]
#b = a["lat"]
#b

In [428]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [429]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[['Lat', 'Lng']]

# Display figure
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))